In [1]:
from urllib import request
import pandas as pd
import numpy as np
import logging
import torch
from collections import Counter
import random

In [3]:
# check gpu
cuda_available = torch.cuda.is_available()
print('Cuda available? ',cuda_available)

Cuda available?  True


In [4]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [8]:
from dont_patronize_me import DontPatronizeMe
import os

directory_path = "./Dont_Patronize_Me_Trainingset/"
dpm = DontPatronizeMe('./Dont_Patronize_Me_Trainingset', '.')
dpm.load_task1()
    
print(dpm.train_task1_df.iloc[0])


par_id                                                        1
art_id                                               @@24942188
keyword                                                hopeless
country                                                      ph
text          We 're living in times of absolute insanity , ...
label                                                         0
orig_label                                                    0
Name: 0, dtype: object


In [9]:
trids = pd.read_csv('./Train_Dev_Split/train_semeval_parids-labels.csv')
teids = pd.read_csv('./Train_Dev_Split/dev_semeval_parids-labels.csv')


In [10]:
trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)

In [11]:
data=dpm.train_task1_df

rows = [] # will contain par_id, label and text
for idx in range(len(trids)):  
  parid = trids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })

trdf1 = pd.DataFrame(rows)

# random.shuffle(trdf1)

trdf1

,par_id,community,text,label
0,4341,poor-families,"The scheme saw an estimated 150,000 children f...",1
1,4136,homeless,Durban 's homeless communities reconciliation ...,1
2,10352,poor-families,The next immediate problem that cropped up was...,1
3,8279,vulnerable,Far more important than the implications for t...,1
4,1164,poor-families,To strengthen child-sensitive social protectio...,1
...,...,...,...,...
8370,8380,refugee,Rescue teams search for survivors on the rubbl...,0
8371,8381,hopeless,The launch of ' Happy Birthday ' took place la...,0
8372,8382,homeless,"The unrest has left at least 20,000 people dea...",0
8373,8383,hopeless,You have to see it from my perspective . I may...,0


In [12]:

rows = [] # will contain par_id, label and text
for idx in range(len(teids)):  
  parid = teids.par_id[idx]
  #print(parid)
  # select row from original dataset
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })
  
tdevf1 = pd.DataFrame(rows)

In [ ]:
import torch
from transformers import RobertaConfig, RobertaModel,RobertaForSequenceClassification
from transformers import RobertaTokenizer
from transformers import AdamW
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from datasets import Dataset
import random 
random.seed(1)

/vol/bitbucket/pg721/nlp_cw/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
configuration = RobertaConfig()
model = RobertaForSequenceClassification(configuration)
configuration = model.config

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained('roberta-base')

train_ds = Dataset.from_pandas(trdf1)
print(train_ds)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Dataset({
    features: ['par_id', 'community', 'text', 'label'],
    num_rows: 8375
})


In [11]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1":f1}

In [12]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])
    
optimizer = AdamW(model.parameters(), lr=5e-5)

X_train = list(train_ds["text"])
y_train = list(train_ds["label"])

X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
train_dataset = Dataset(X_train_tokenized, y_train)
model.zero_grad()

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    compute_metrics=compute_metrics
)


/vol/bitbucket/pg721/nlp_cw/venv/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss
500,0.336300
1000,0.337000
1500,0.335800
2000,0.318500
2500,0.318700
